In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
import pickle

with open("cleaned_df_features.pkl", "rb") as f:
    df, numerics, curated_cat, other_cat = pickle.load(f)

In [3]:
# first we split trials

from sklearn.model_selection import train_test_split, KFold

# 10% for testing
[df_full_train,df_test] = train_test_split(df,test_size=0.1,random_state=42)
# 72% and 18% for train/val
[df_train,df_val] = train_test_split(df_full_train,test_size=0.2,random_state=42)
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

(36000, 14)
(9000, 14)
(5000, 14)


In [4]:
# get response variable out

y_test = df_test.base_passenger_fare.values
y_val = df_val.base_passenger_fare.values
y_train = df_train.base_passenger_fare.values

df_test = df_test.drop(columns=['base_passenger_fare'])
df_val = df_val.drop(columns=['base_passenger_fare'])
df_train = df_train.drop(columns=['base_passenger_fare'])

### starting with the curated feature set

In [5]:
# one-hot encoding: fit from training set

from sklearn.feature_extraction import DictVectorizer
import time

dv_cur = DictVectorizer(sparse=True)


# curated features only
X_cur_train = dv_cur.fit_transform(df_train[numerics + curated_cat].to_dict(orient='records'))
X_cur_val = dv_cur.transform(df_val[numerics + curated_cat].to_dict(orient='records'))

print(X_cur_train.shape)
    
dv_cur.get_feature_names_out()

(36000, 40)


array(['day_of_week=Friday', 'day_of_week=Monday', 'day_of_week=Saturday',
       'day_of_week=Sunday', 'day_of_week=Thursday',
       'day_of_week=Tuesday', 'day_of_week=Wednesday', 'hour_of_day=0',
       'hour_of_day=1', 'hour_of_day=10', 'hour_of_day=11',
       'hour_of_day=12', 'hour_of_day=13', 'hour_of_day=14',
       'hour_of_day=15', 'hour_of_day=16', 'hour_of_day=17',
       'hour_of_day=18', 'hour_of_day=19', 'hour_of_day=2',
       'hour_of_day=20', 'hour_of_day=21', 'hour_of_day=22',
       'hour_of_day=23', 'hour_of_day=3', 'hour_of_day=4',
       'hour_of_day=5', 'hour_of_day=6', 'hour_of_day=7', 'hour_of_day=8',
       'hour_of_day=9', 'hvfhs_license_num=Juno',
       'hvfhs_license_num=Lyft', 'hvfhs_license_num=Uber',
       'hvfhs_license_num=Via', 'shared_flag_or', 'trip_miles_log1p',
       'trip_time_log1p', 'wait_time_sec_log1p', 'wav_request_flag'],
      dtype=object)

In [6]:
ftns_cur = dv_cur.get_feature_names_out()
ftns_cur[[-2,-3,-4]]

array(['wait_time_sec_log1p', 'trip_time_log1p', 'trip_miles_log1p'],
      dtype=object)

In [7]:


from sklearn.metrics import root_mean_squared_error
from sklearn.linear_model import Ridge, Lasso
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Optional: progress bar
from tqdm.auto import tqdm

print("\n||||||| curated features |||||||\n")

# ------------------------------------
# 1. Linear models: L2 (Ridge) search
# ------------------------------------

# Note: with one-hot/sparse features, use with_mean=False
# If X_* are sparse, this will keep them sparse where possible.


# only scale 3 numeric features
numeric_idx = [36,37,38]
all_idx = list(range(40))
cat_idx = [j for j in all_idx if j not in numeric_idx]

preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(with_mean=False), numeric_idx),
        ("cat", "passthrough", cat_idx),
    ]
)


ridge_alphas = [0.0, 0.1, 1.0, 10.0, 100.0]

print("\n=== Ridge (L2) hyperparameter search ===")

ridge_results = []
for alpha in tqdm(ridge_alphas, desc="Ridge alphas"):
    model_ridge = Pipeline([
        ("scaler", preprocess),  # safe for sparse
        ("reg", Ridge(alpha=alpha, random_state=0))
    ])

    t0 = time.time()
    model_ridge.fit(X_cur_train, y_train)

    y_pred = model_ridge.predict(X_cur_train)
    rmse_train = root_mean_squared_error(y_train, y_pred)
    
    y_pred = model_ridge.predict(X_cur_val)
    rmse_val = root_mean_squared_error(y_val, y_pred)
    
    t1 = time.time()
    train_time = t1 - t0

    print(f"alpha={alpha:8.4f} | train_RMSE={rmse_train:8.4f} | val_RMSE={rmse_val:8.4f} | time={train_time/60:5.2f} min")
    ridge_results.append((alpha, rmse_train, rmse_val, train_time, model_ridge))

# Pick best Ridge by validation RMSE
best_ridge_alpha, best_ridge_rmse_train, best_ridge_rmse_val, best_ridge_time, best_ridge_model = min(
    ridge_results,
    key=lambda x: x[2]
)

print(f"\nBest Ridge alpha: {best_ridge_alpha}")
print(f"Best Ridge training RMSE: {best_ridge_rmse_train:.4f}")
print(f"Best Ridge validation RMSE: {best_ridge_rmse_val:.4f}")
print(f"Best Ridge training time: {best_ridge_time/60:.2f} minutes")


||||||| curated features |||||||


=== Ridge (L2) hyperparameter search ===


Ridge alphas:   0%|          | 0/5 [00:00<?, ?it/s]

alpha=  0.0000 | train_RMSE=  9.7020 | val_RMSE=  9.6686 | time= 0.00 min
alpha=  0.1000 | train_RMSE=  9.7020 | val_RMSE=  9.6686 | time= 0.00 min
alpha=  1.0000 | train_RMSE=  9.7020 | val_RMSE=  9.6686 | time= 0.00 min
alpha= 10.0000 | train_RMSE=  9.7020 | val_RMSE=  9.6688 | time= 0.00 min
alpha=100.0000 | train_RMSE=  9.7031 | val_RMSE=  9.6716 | time= 0.00 min

Best Ridge alpha: 0.0
Best Ridge training RMSE: 9.7020
Best Ridge validation RMSE: 9.6686
Best Ridge training time: 0.00 minutes


In [8]:
y_train_pred_baseline = np.full_like(y_train, np.mean(y_train), dtype=float)
y_val_pred_baseline = np.full_like(y_val, np.mean(y_train), dtype=float)

print("Baseline train RMSE:", root_mean_squared_error(y_train, y_train_pred_baseline))
print("Baseline valid RMSE:", root_mean_squared_error(y_val, y_val_pred_baseline))

Baseline train RMSE: 15.771902985738684
Baseline valid RMSE: 16.110006426103364


In [9]:
from sklearn.linear_model import SGDRegressor


# only scale 3 numeric features
numeric_idx = [36,37,38]
all_idx = list(range(40))
cat_idx = [j for j in all_idx if j not in numeric_idx]

preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(with_mean=False), numeric_idx),
        ("cat", "passthrough", cat_idx),
    ]
)


SGDalphas = [1e-5, 1e-4, 1e-3, 1e-2]
l1_ratios = [0.0, 0.05, 0.1, 0.2, 0.5]

print("\n=== ElasticNet hyperparameter search ===")
SGD_results = []

for alpha in tqdm(SGDalphas, desc="SGD ElasticNet alphas"):
    for l1r in tqdm(l1_ratios, desc="l1 ratio", leave=False):
    
        model_SGD = Pipeline([
            ("scaler", preprocess),
            ("reg", SGDRegressor( \
                loss="squared_error",
                penalty="elasticnet",
                alpha=alpha,
                l1_ratio=l1r,
                eta0=1e-6,
                max_iter=100,
                shuffle=True,
                early_stopping=True,
                n_iter_no_change=5,
                validation_fraction=0.1,
                tol=1e-3,
                verbose=0)
            )
        ])
        
        t0 = time.time()
        model_SGD.fit(X_cur_train, y_train)
    
        y_pred = model_SGD.predict(X_cur_train)
        rmse_train = root_mean_squared_error(y_train, y_pred)
        
        y_pred = model_SGD.predict(X_cur_val)
        rmse_val = root_mean_squared_error(y_val, y_pred)
        
        t1 = time.time()
        train_time = t1 - t0
        
        print(f"alpha={alpha:8.6f} | l1r={l1r:8.4f} | train_RMSE={rmse_train:8.4f} | val_RMSE={rmse_val:8.4f} | time={train_time/60:5.2f} min")
        SGD_results.append((alpha, l1r, rmse_train, rmse_val, train_time, model_SGD))


best_SGD_alpha, best_SGD_l1r, best_SGD_rmse_train, best_SGD_rmse_val, best_SGD_time, best_SGD_model = min(
    SGD_results,
    key=lambda x: x[3]
)

print(f"\nBest SGD alpha: {best_SGD_alpha}")
print(f"Best SGD L1 ratio: {best_SGD_l1r:.4f}")
print(f"Best SGD training RMSE: {best_SGD_rmse_train:.4f}")
print(f"Best SGD validation RMSE: {best_SGD_rmse_val:.4f}")
print(f"Best SGD training time: {best_SGD_time/60:.2f} minutes")


=== ElasticNet hyperparameter search ===


SGD ElasticNet alphas:   0%|          | 0/4 [00:00<?, ?it/s]

l1 ratio:   0%|          | 0/5 [00:00<?, ?it/s]

alpha=0.000010 | l1r=  0.0000 | train_RMSE= 14.7783 | val_RMSE= 15.0946 | time= 0.00 min
alpha=0.000010 | l1r=  0.0500 | train_RMSE= 14.7548 | val_RMSE= 15.0706 | time= 0.00 min
alpha=0.000010 | l1r=  0.1000 | train_RMSE= 14.6588 | val_RMSE= 14.9693 | time= 0.00 min
alpha=0.000010 | l1r=  0.2000 | train_RMSE= 14.7884 | val_RMSE= 15.1047 | time= 0.00 min
alpha=0.000010 | l1r=  0.5000 | train_RMSE= 14.7840 | val_RMSE= 15.1008 | time= 0.00 min


l1 ratio:   0%|          | 0/5 [00:00<?, ?it/s]

alpha=0.000100 | l1r=  0.0000 | train_RMSE= 14.7243 | val_RMSE= 15.0384 | time= 0.00 min
alpha=0.000100 | l1r=  0.0500 | train_RMSE= 14.7486 | val_RMSE= 15.0642 | time= 0.00 min
alpha=0.000100 | l1r=  0.1000 | train_RMSE= 14.7128 | val_RMSE= 15.0258 | time= 0.00 min
alpha=0.000100 | l1r=  0.2000 | train_RMSE= 14.8089 | val_RMSE= 15.1272 | time= 0.00 min
alpha=0.000100 | l1r=  0.5000 | train_RMSE= 14.7540 | val_RMSE= 15.0683 | time= 0.00 min


l1 ratio:   0%|          | 0/5 [00:00<?, ?it/s]

alpha=0.001000 | l1r=  0.0000 | train_RMSE= 14.7692 | val_RMSE= 15.0847 | time= 0.00 min
alpha=0.001000 | l1r=  0.0500 | train_RMSE= 14.7046 | val_RMSE= 15.0173 | time= 0.00 min
alpha=0.001000 | l1r=  0.1000 | train_RMSE= 14.7233 | val_RMSE= 15.0375 | time= 0.00 min
alpha=0.001000 | l1r=  0.2000 | train_RMSE= 14.6764 | val_RMSE= 14.9881 | time= 0.00 min
alpha=0.001000 | l1r=  0.5000 | train_RMSE= 14.7143 | val_RMSE= 15.0274 | time= 0.00 min


l1 ratio:   0%|          | 0/5 [00:00<?, ?it/s]

alpha=0.010000 | l1r=  0.0000 | train_RMSE= 14.8155 | val_RMSE= 15.1333 | time= 0.00 min
alpha=0.010000 | l1r=  0.0500 | train_RMSE= 14.6835 | val_RMSE= 14.9949 | time= 0.00 min
alpha=0.010000 | l1r=  0.1000 | train_RMSE= 14.7271 | val_RMSE= 15.0409 | time= 0.00 min
alpha=0.010000 | l1r=  0.2000 | train_RMSE= 14.7739 | val_RMSE= 15.0901 | time= 0.00 min
alpha=0.010000 | l1r=  0.5000 | train_RMSE= 14.7189 | val_RMSE= 15.0318 | time= 0.00 min

Best SGD alpha: 1e-05
Best SGD L1 ratio: 0.1000
Best SGD training RMSE: 14.6588
Best SGD validation RMSE: 14.9693
Best SGD training time: 0.00 minutes


### summary for Ridge and SGD Elastic Net regression with curated features

Ridge trains very fast (<1min) and SGD is a lot slower (a few minutes per model and more hyperparams). 

Both had similar performances, with ridge being slightly better (RMSE= 9.67 vs 14.88). 

(This is not what I expected. I think it might be because I did not pick the best hyperparams for SGD and did not let it train for very long.) 

Both training and validation RMSE are large, although they are better than the baseline model of mean (RMSE=16). 

Given the median fare is $10, the model is completely underfitting. 

Maybe the curated features are not informative enough, or the linear regression model is too biased, or maybe RMSE is not a good metric for training because of the skewness of our target variable.

In any case, I will try XGBoost next. Hopefully a more flexible model will give us better performance.

In [10]:

from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint


# -------------------------------
# 3. XGBoost: randomized search
# -------------------------------

# portion out some data from _train for early stopping
X_temp, X_stop_xgb, y_temp, y_stop_xgb = train_test_split(
    X_cur_train, y_train, test_size=0.1, random_state=42)
# use fewer data for training for speed
X_temp, X_train_xgb, y_temp, y_train_xgb = train_test_split(
    X_temp, y_temp, test_size=0.2, random_state=42)

print(f"eval (early stopping) on {y_stop_xgb.shape[0]:d} rows")
print(f"train (CV tuning) on {y_train_xgb.shape[0]:d} rows")

# Define parameter distribution
param_dist = {
    'max_depth': [3,4,5,6,7,8],
    'learning_rate': [0.3, 0.2, 0.1, 0.05, 0.01],
    'min_child_weight': [1, 5, 10, 20],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

# Create XGBClassifier
xgb = XGBRegressor(
    tree_method="hist",
    enable_categorical=True,  # if using pandas categorical dtypes
    n_estimators=500,        # large, rely on early stopping
    objective="reg:squarederror",
    eval_metric="rmse",
    early_stopping_rounds=10,
    n_jobs=-1
)

fit_params = {
    "eval_set": [(X_stop_xgb, y_stop_xgb)],
    "verbose": False,
}


search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_dist,
    n_iter=50,          # e.g. 50 random trials
    scoring="neg_root_mean_squared_error",
    verbose=2,          # shows progress of the search
    n_jobs=1,
    cv=3   
)

print("\n=== XGBoost hyperparameter search ===")
t0 = time.time()
search.fit(X_train_xgb, y_train_xgb, **fit_params)
t1 = time.time()




print("XGB Best params:", search.best_params_)
print("XGB Best CV score (RMSE):", -search.best_score_)
print(f"XGB search time: {(t1 - t0)/60:.2f} minutes")

best_xgb = search.best_estimator_

    
# Evaluate best XGB on validation set explicitly:
y_pred_xgb = best_xgb.predict(X_cur_train)
xgb_train_rmse = root_mean_squared_error(y_train, y_pred_xgb)

y_pred_xgb = best_xgb.predict(X_cur_val)
xgb_val_rmse = root_mean_squared_error(y_val, y_pred_xgb)
print(f"XGB training RMSE (best model): {xgb_train_rmse:.4f}")
print(f"XGB validation RMSE (best model): {xgb_val_rmse:.4f}")



# ------------------------------------
# 4. Summary of model comparison
# ------------------------------------

#print("\n=== Summary (validation RMSE) ===")
#print(f"Ridge (L2)   : {best_ridge_rmse:.4f}  (alpha={best_ridge_alpha})")
#print(f"XGBoost      : {xgb_val_rmse:.4f}")

eval (early stopping) on 3600 rows
train (CV tuning) on 6480 rows

=== XGBoost hyperparameter search ===
Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV] END colsample_bytree=0.6, learning_rate=0.1, max_depth=7, min_child_weight=1, subsample=1.0; total time=   0.3s
[CV] END colsample_bytree=0.6, learning_rate=0.1, max_depth=7, min_child_weight=1, subsample=1.0; total time=   0.1s
[CV] END colsample_bytree=0.6, learning_rate=0.1, max_depth=7, min_child_weight=1, subsample=1.0; total time=   0.1s
[CV] END colsample_bytree=1.0, learning_rate=0.3, max_depth=3, min_child_weight=20, subsample=0.6; total time=   0.0s
[CV] END colsample_bytree=1.0, learning_rate=0.3, max_depth=3, min_child_weight=20, subsample=0.6; total time=   0.0s
[CV] END colsample_bytree=1.0, learning_rate=0.3, max_depth=3, min_child_weight=20, subsample=0.6; total time=   0.0s
[CV] END colsample_bytree=0.6, learning_rate=0.1, max_depth=4, min_child_weight=1, subsample=1.0; total time=   0.1s
[CV] END co

In [11]:
# let's check the results
results = search.cv_results_
df_results = pd.DataFrame({
    "mean_fit_time": results["mean_fit_time"],
    "param_subsample": results["param_subsample"],
    "param_min_child_weight": results["param_min_child_weight"],
    "param_max_depth": results["param_max_depth"],
    "param_learning_rate": results["param_learning_rate"],
    "param_colsample_bytree": results["param_colsample_bytree"],
    "mean_CV_RMSE": -results["mean_test_score"],
    "std_test_score": results["std_test_score"],
    "rank": results["rank_test_score"],
})

df_results.sort_values("rank",inplace=True)
df_results

,mean_fit_time,param_subsample,param_min_child_weight,param_max_depth,param_learning_rate,param_colsample_bytree,mean_CV_RMSE,std_test_score,rank
42,0.076635,1.0,1,4,0.20,0.8,7.423796,1.616016,1
0,0.240340,1.0,1,7,0.10,0.6,7.470099,1.701736,2
2,0.142995,1.0,1,4,0.10,0.6,7.471871,1.631738,3
23,0.118572,0.6,1,4,0.10,1.0,7.506514,1.769289,4
10,0.145603,1.0,1,7,0.10,0.8,7.532734,1.660957,5
28,0.176727,0.6,1,5,0.05,0.8,7.535577,1.667076,6
43,0.311408,0.8,1,7,0.05,0.6,7.556295,1.636969,7
45,0.069810,1.0,1,3,0.30,1.0,7.570933,1.618129,8
33,0.266548,1.0,1,3,0.05,0.6,7.583824,1.679783,9
29,0.072170,0.6,1,6,0.20,1.0,7.590988,1.845288,10


In [12]:
df_results["mean_CV_RMSE"].describe()

count    50.000000
mean      7.675918
std       0.114770
min       7.423796
25%       7.604401
50%       7.674233
75%       7.736918
max       8.031789
Name: mean_CV_RMSE, dtype: float64

### summary for XGBoost with curated features
XGBoost's RMSE is stuck very closely around $6.5 for both train and val. Hyperparameters make little difference to performance. This suggest underfitting. Because XGBoost is very powerful and flexible, I think the limiting factor is the curated features. It's time to use all features.